In [1]:
from lyricsgenius import Genius
import pandas as pd
import os
import re
import requests
import sys
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI
from bs4 import BeautifulSoup
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter 
import urllib.request
import json
import pickle
import h5py
import csv
import os
from bs4 import BeautifulSoup
# from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'lyricsgenius'

In [15]:
GENIUS_API_TOKEN = os.getenv('GENIUS_API_TOKEN')
viper_id = '48746'
genius = Genius(GENIUS_API_TOKEN)

# # Search for the artist
# artist = genius.search_artist("Viper", max_songs=None, sort='popularity')

# # Access the songs attribute of the artist to get a list of their songs
# songs = artist.songs

In [16]:
song_title = "Shape of You"
artist_name = "Ed Sheeran"

song = genius.search_song(song_title)

# Print the song lyrics
if song:
    print(f"Lyrics for '{song.title}' by {song.artist}:\n")
    print(song.lyrics)
else:
    print("Song not found!")

Searching for "Shape of You"...
Done.
Lyrics for 'Shape of You' by Ed Sheeran:

[Verse 1]
A club isn't the best place to find a lover
So the bar is where I go (Mm)
Me and my friends at the table doin' shots
Drinkin' fast and then we talk slow (Mm)
And you come over and start up a conversation with just me
And trust me, I'll give it a chance now (Mm)
Take my hand, stop, put Van the Man on the jukebox
And then we start to dance, and now I'm singin' like

[Pre-Chorus]
Girl, you know I want your love
Your love was handmade for somebody like me
Well, come on now, follow my lead
I may be crazy, don't mind me
Say, "Boy, let's not talk too much"
Grab on my waist and put that body on me
Well, come on now, follow my lead
Come, come on now, follow my lead, mm

[Chorus]
I'm in love with the shape of you
We push and pull like a magnet do
Although my heart is fallin' too
I'm in love with your body
And last night, you were in my room
And now my bed sheets smell like you
Every day, discoverin' somethi

In [21]:
'''
SONG OBJECT - 
Song.title - Title of the song.
Song.artist - Primary artist on the song.
Song.lyrics - Full set of song lyrics.
Song.album - Name of the album the song is on.
Song.year - Year the song was released.
Song.url - URL to the song on Genius.
Song.album_url - URL to the song album.
Song.featured_artists - Artists featured on the song.
Song.producer_artists - Producers of the song.
Song.media - External IDs of the song.
Song.writer_artists - List of artists credited as writers.
Song.song_art_image_url - URL to the song’s cover art.
'''
titles = []
lyrics = []

for song in songs:
    titles.append(song.title)
    lyrics.append(song.lyrics[15:])

# Step 3: Create a DataFrame
df_viper = pd.DataFrame({
    "Title": titles,
    "Lyrics": lyrics
})
    
df_viper.to_csv('viper_songs.csv', index=False, encoding='utf-8')


NameError: name 'songs' is not defined

In [18]:
# Read the CSV file back into a DataFrame
df_viper = pd.read_csv('viper_songs.csv', encoding='utf-8')

# Display the DataFrame to verify it was read correctly
print(df_viper.head())

                                    Title  \
0   You’ll Cowards Don’t Even Smoke Crack   
1  You Wanna See Me Dead Cause Of My Hops   
2  Hey, Maybe One Day You’ll See Me Again   
3                         I Sell Dope Boy   
4                            You Gone Die   

                                              Lyrics  
0  You’ll Cowards Don’t Even Smoke Crack Lyrics[I...  
1  You Wanna See Me Dead Cause Of My Hops Lyrics[...  
2  Hey, Maybe One Day You’ll See Me Again Lyrics[...  
3  I Sell Dope Boy Lyrics[Chorus]\nWhat's up, I d...  
4  You Gone Die Lyrics[Chorus]\nYou gon' die\nYou...  


### Generate Embeddings

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.85s/it]
/Users/Nev/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v2/5130cf1daf847c1bacee854a6ef1ca939e747fb2/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/opt/anaconda3/lib/python3.12/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


[[87.42694854736328, 0.46282508969306946], [0.9652770757675171, 86.03724670410156]]


## Setting up Emotion Dataset Embedding

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True)

documents = ["I feel great today", "I feel sad today", "I feel so so today", "I feel happy today", "I feel angry today"]

document_embeddings = model.encode(documents)
model.max_seq_length = 8192
## model.encode(list)
print(document_embeddings)

AttributeError: 'Qwen2ForCausalLM' object has no attribute 'encode'

### Embed the emotion dataset

In [31]:
import pandas as pd

splits = {'train': 'split/train-00000-of-00001.parquet', 'validation': 'split/validation-00000-of-00001.parquet', 'test': 'split/test-00000-of-00001.parquet'}
df_emotion = pd.read_parquet("hf://datasets/dair-ai/emotion/" + splits["train"])
df_emotion['embedding'] = df_emotion['text'].apply(lambda x: model.encode(x).tolist())

In [32]:
df_emotion.to_csv('df_emotion_with_embeddings.csv', index=False, encoding='utf-8')

In [45]:
df_viper = pd.read_csv('viper_songs.csv', encoding='utf-8')
df_viper['Embedding'] = df_viper['Lyrics'].apply(lambda x: model.encode(x).tolist())
print(df_viper.head())

                                    Title  \
0   You’ll Cowards Don’t Even Smoke Crack   
1  You Wanna See Me Dead Cause Of My Hops   
2  Hey, Maybe One Day You’ll See Me Again   
3                         I Sell Dope Boy   
4                            You Gone Die   

                                              Lyrics  \
0  You’ll Cowards Don’t Even Smoke Crack Lyrics[I...   
1  You Wanna See Me Dead Cause Of My Hops Lyrics[...   
2  Hey, Maybe One Day You’ll See Me Again Lyrics[...   
3  I Sell Dope Boy Lyrics[Chorus]\nWhat's up, I d...   
4  You Gone Die Lyrics[Chorus]\nYou gon' die\nYou...   

                                           Embedding  
0  [-0.028602279722690582, 0.030411386862397194, ...  
1  [-0.02250194363296032, 0.031217927113175392, -...  
2  [-0.03493272140622139, 0.014527689665555954, -...  
3  [-0.010006560012698174, 0.02911408245563507, -...  
4  [-0.05293525010347366, 0.051491811871528625, -...  


In [51]:
print(df_emotion.head())

                                                text  label  \
0                            i didnt feel humiliated      0   
1  i can go from feeling so hopeless to so damned...      0   
2   im grabbing a minute to post i feel greedy wrong      3   
3  i am ever feeling nostalgic about the fireplac...      2   
4                               i am feeling grouchy      3   

                                           embedding  
0  [-0.010654906742274761, 0.02735527791082859, 0...  
1  [0.018200771883130074, 0.03985970467329025, 0....  
2  [-0.04588513448834419, 0.029571833088994026, 0...  
3  [-0.020895397290587425, -0.04290849342942238, ...  
4  [-0.0029021133668720722, -0.001833997666835785...  


In [62]:
from sklearn.metrics.pairwise import cosine_similarity

max_similarities = []  # Store max similarities for each row in df_viper
closest_matches_of_text = []  # Store closest matches from df_emotion for each row in df_viper
closest_matches_of_labels = []  # Store closest matches from df_emotion for each row in df_viper

# Iterate over rows in df_viper
for viper_index, viper_row in df_viper.iterrows():
    viper_embedding = np.array(viper_row['Embedding']).reshape(1, -1)
    print('viper indx is', viper_index)
    max_similarity = -1  # Initialize max similarity
    closest_match = None  # Initialize closest match
    
    # Iterate over rows in df_emotion
    for emotion_index, emotion_row in df_emotion.iterrows():
        emotion_embedding = np.array(emotion_row['embedding']).reshape(1, -1)
        
        # Calculate cosine similarity between current viper and emotion embeddings
        similarity = cosine_similarity(viper_embedding, emotion_embedding)[0][0]
        
        # Update if we found a higher similarity
        if similarity > max_similarity:
            max_similarity = similarity
            closest_match_text = emotion_row['text']  # Assuming you're interested in the 'Title' of the matching row
            closest_match_label = emotion_row['label']  # Assuming you're interested in the 'Title' of the matching row
    
    # Store the closest match and max similarity for the current row in df_viper
    max_similarities.append(max_similarity)
    closest_matches_of_text.append(closest_match_text)
    closest_matches_of_labels.append(closest_match_label)

# Add results to df_viper
df_viper['Closest Match Text'] = closest_matches_of_text
df_viper['Closest Match Emotion'] = closest_match_label
df_viper['Max Similarity'] = max_similarities

# Print the result
print(df_viper[['Title', 'Closest Match', 'Max Similarity']])

viper indx is 0
viper indx is 1
viper indx is 2
viper indx is 3
viper indx is 4
viper indx is 5
viper indx is 6
viper indx is 7
viper indx is 8
viper indx is 9
viper indx is 10
viper indx is 11
viper indx is 12
viper indx is 13
viper indx is 14
viper indx is 15
viper indx is 16
viper indx is 17
viper indx is 18
viper indx is 19
viper indx is 20
viper indx is 21
viper indx is 22
viper indx is 23
viper indx is 24
viper indx is 25
viper indx is 26
viper indx is 27
viper indx is 28
viper indx is 29
viper indx is 30
viper indx is 31
viper indx is 32
viper indx is 33
viper indx is 34
viper indx is 35
viper indx is 36
viper indx is 37
viper indx is 38
viper indx is 39
viper indx is 40
viper indx is 41
viper indx is 42
viper indx is 43
viper indx is 44
viper indx is 45
viper indx is 46
viper indx is 47
viper indx is 48
viper indx is 49
viper indx is 50
viper indx is 51
viper indx is 52
viper indx is 53
viper indx is 54
viper indx is 55
viper indx is 56
viper indx is 57
viper indx is 58
viper i

In [56]:
df_viper.to_csv('df_viper_with_best_match.csv', index=False, encoding='utf-8')

In [65]:
count = 0
for row, col in df_viper.iterrows():
    print(col['Title'])
    print(col['Closest Match Text'])
    #, col['Closest Match Emotion'], col['Max Similarity'])
    print('---' * 50)
    if count == 5:
        break
    count += 1

You’ll Cowards Don’t Even Smoke Crack
i want to be recless but im feeling so uptight put your mamma in a headlock baby and do it right whooooos got the crack whooooooooos got the crack whooooo s got the crack whos got the craaaaaaack
------------------------------------------------------------------------------------------------------------------------------------------------------
You Wanna See Me Dead Cause Of My Hops
i want to be recless but im feeling so uptight put your mamma in a headlock baby and do it right whooooos got the crack whooooooooos got the crack whooooo s got the crack whos got the craaaaaaack
------------------------------------------------------------------------------------------------------------------------------------------------------
Hey, Maybe One Day You’ll See Me Again
i cant let all these feeling in one blink im not a heartless person like you i give you a lot of change i give you a lot of change to come and change your decisions i give you a lot of chang

In [ ]:
"""
0 = Sadness
1 = Joy
2 = Love
3 = Anger
4 = Fear
5 = Surprise
"""

## All music dataset

In [21]:
q1_words = ['aroused', 'astonished', 'excited', 'delighted', 'happy']
q2_words = ['alarmed', 'tense', 'angry', 'afraid', 'annoyed', 'distressed', 'frustrated']
q3_words = ['misreable', 'sad', 'depressed', 'gloomy', 'bored', 'droopy', 'tired']
q4_words = ['pleased', 'glad', 'serene', 'content', 'at ease', 'satisfied', 'relaxed', 'calm']
class_to_words = {
    'Q1': q1_words,
    'Q2': q2_words,
    'Q3': q3_words,
    'Q4': q4_words
}

df_all_music = pd.read_csv('Allmusic_dataset.csv', encoding='utf-8')
df_all_music['Mapped_Words'] = df_all_music['Class'].map(class_to_words)

def find_lyrics(artist, title):
    title = title.replace('_', ' ')
    artist = artist.replace('_', ' ')
    
    try:
        song = genius.search_song(f'{title} {artist}')
    except:
        return None
    
    if song:
        return song.lyrics
    else:
        print(f"Lyrics for '{title}' by {artist} not found!")
        return None

df_all_music['Lyrics'] = df_all_music.apply(lambda x: find_lyrics(x['Artist'], x['Title']), axis=1)
print(df_all_music.head())
df_all_music.to_csv('Allmusic_dataset_with_lyrics.csv', index=False, encoding='utf-8')

Searching for "You're Too Lovely to Last Billie Holiday"...
Done.
Searching for "Adios Amigo Jim Reeves"...
Done.
Searching for "Concrete Jungle Bob Marley"...
Done.
Searching for "Till the End of Time Perry Como"...
Done.
Searching for "Uncertain Weather Genesis"...
Done.
Searching for "Crazy He Calls Me Billie Holiday"...
Done.
Searching for "Without Your Love Billie Holiday"...
Done.
Searching for "Beer Barrel Polka The Andrews Sisters"...
Done.
Searching for "The Foetus of a New Day Kicking Cradle of Filth"...
Done.
Searching for "God Won't Get You Dolly Parton"...
Done.
Searching for "They Are All in Love The Who"...
Done.
Searching for "“Green Grass of Home Elvis Presley"...
Done.
Searching for "Getcha Groove On Limp Bizkit"...
Done.
Searching for "How Sweet It Is (To Be Loved by You) Marvin Gaye"...
Done.
Searching for "Breathe Blu Cantrell"...
Done.
Searching for "Mexicali Rose Gene Autry"...
Done.
Searching for "You Ain't Goin' Nowhere The Byrds"...
Done.
Searching for "Terrap

In [4]:
model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True)
df_all_music = pd.read_csv('Allmusic_dataset_with_lyrics.csv', encoding='utf-8')
df_viper = pd.read_csv('viper_songs.csv', encoding='utf-8')
df_all_music = df_all_music.dropna(subset=['Lyrics'])

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.65it/s]


In [21]:

q1_lyrics = df_all_music[df_all_music['Class'] == 'Q1']['Lyrics'].tolist()
q2_lyrics = df_all_music[df_all_music['Class'] == 'Q2']['Lyrics'].tolist()
q3_lyrics = df_all_music[df_all_music['Class'] == 'Q3']['Lyrics'].tolist()
q4_lyrics = df_all_music[df_all_music['Class'] == 'Q4']['Lyrics'].tolist()

lyrics_list = [q1_lyrics, q2_lyrics, q3_lyrics, q4_lyrics]
lyrics_embedding_dict = {}

for lyrics_set in lyrics_list:
    print(f"on lyrics set {lyrics_set}")
    embeddings = model.encode(lyrics_set, batch_size=5)  # Reduce batch size to 8 or smaller
    avg_embedding = np.mean(embeddings, axis=0)
    
    # Store the average embedding in the dictionary
    lyrics_embedding_dict[str(lyrics_set)] = avg_embedding

    print(f"Average embedding for this set: {avg_embedding[:5]}...")
print(q1_lyrics)

on lyrics set ["In the navy\nIn the navy\nEvery navy mans the right guy\nEveryone gives him the bright eye\nAnd the navy they do right by\n\nTo a sailor\nTo a sailor\nAll the world is just his main street\nWhen you're in the navy\nYou're in the worlds best gleam\n\nFirst you take your training as a boot\nSwing a hammock navigate & shoot\nJump open ships & subs & planes\nThe seven seas are open lane\n\nWhen the bow is down with Davy Jones\nFolks will welcome home their rolling stones\nFor every navy mans a king\nAnd you'll hear everybody sing\n\nHere comes the navy\nBreak out the red, white & blue\nHere comes the navy\nFighting & sea going through\nMake way for navy\nNavy with anchors aweigh\nEverybody loves the navy of the usa\n\nFirst you take your training as a boot\nSwing a hammock & navigate & shoot\nJump open ships & in subs & in planes\nThe seven seas are open lane\n\nHere it comes, here it comes, here comes the navy\nSing a song of good cheer because the navy is here\nEverybody 

In [22]:
np.save('average_lyrics_embeddings.npy', lyrics_embedding_dict)  # Save the embeddings dictionary

In [24]:
loaded_lyrics_embedding_dict = np.load('average_lyrics_embeddings.npy', allow_pickle=True).item()

In [27]:
testt = df_viper.iloc[0]['Lyrics']
embedding = model.encode(testt)
print(len(embedding))

1536


In [15]:
def embed_lyrics(lyrics):
    embedding = model.encode(lyrics)
    print(len(embedding))
    return embedding
df_viper["Embedding"] = df_viper["Lyrics"].apply(embed_lyrics)

1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536


In [16]:
embeddings = df_viper["Embedding"].tolist()

# Convert to numpy array (if you want to store it as a structured array)
embeddings_np = np.array(embeddings)

# Save embeddings to .npy file
np.save('viper_songs_embeddings.npy', embeddings_np)

In [35]:
import numpy as np
import pandas as pd
loaded_embeddings = np.load('viper_songs_embeddings.npy')

# Print the shape to verify
print(f"Loaded embeddings shape: {loaded_embeddings.shape}")

Loaded embeddings shape: (308, 1536)


In [36]:
df_viper = pd.read_csv('viper_songs.csv', encoding='utf-8')
df_viper["Embedding"]= list(loaded_embeddings)
print(len(df_viper.iloc[0]['Embedding']))

1536


In [59]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the lyrics embedding dictionary
lyrics_embedding_dict = np.load('average_lyrics_embeddings.npy', allow_pickle=True).item()

# Function to compute cosine similarities
def compute_cosine_similarities(song_embedding, lyrics_embedding_dict):
    """Compute the cosine similarities between song embedding and the dictionary of embeddings."""
    cosine_similarities = {}
    
    count = 0
    for key, dict_embedding in lyrics_embedding_dict.items():
        # Compute cosine similarity between the given embedding and the dictionary's embedding
        sim = cosine_similarity([song_embedding], [dict_embedding])[0][0]
        # Convert similarity to percentage
        percentage_similarity = sim * 100
        
        # Store the percentage similarity
        count += 1
        cosine_similarities[f'Q{count}'] = percentage_similarity
    return cosine_similarities

# Loop over df_viper DataFrame (adjust column names accordingly)
similarities_list = []

# Loop over the rows in df_viper
for index, row in df_viper.iterrows():
    embedding = row['Embedding']  # Adjust this depending on how the embedding is stored
    song_title = row['Title']  # Save the song title for reference
    
    # Compute cosine similarities for this song
    cosine_similarities = compute_cosine_similarities(embedding, lyrics_embedding_dict)
    
    # Create a dictionary with the song title and its similarities
    similarities_data = {'Song Title': song_title, 'Cosine Similarities': cosine_similarities}
    
    # Append the data to the list
    similarities_list.append(similarities_data)

similarities_df = pd.DataFrame(similarities_list)
cosine_similarities_expanded = similarities_df['Cosine Similarities'].apply(pd.Series)
df_viper_with_similarities = pd.concat([df_viper, cosine_similarities_expanded], axis=1)
q_columns = ['Q1', 'Q2', 'Q3', 'Q4']

offset = 0.001

for col in q_columns:
    col_min = df_viper_with_similarities[col].min()
    col_max = df_viper_with_similarities[col].max()
    
    # Add offset to the min and max values if the range is zero
    if col_max - col_min == 0:
        col_min -= offset  # Adjust minimum value by a small offset
    
    # Perform Min-Max scaling with a small offset to avoid division by zero
    df_viper_with_similarities[col] = (df_viper_with_similarities[col] - col_min) / (col_max - col_min + offset)

def get_quartile(row):
    max_value = row[['Q1', 'Q2', 'Q3', 'Q4']].max()  # Find the max value in the row across the Q1, Q2, Q3, Q4 columns
    if max_value == row['Q1']:
        return 'Q1'
    elif max_value == row['Q2']:
        return 'Q2'
    elif max_value == row['Q3']:
        return 'Q3'
    elif max_value == row['Q4']:
        return 'Q4'

# Apply the function row by row to assign the Quartile column
df_viper_with_similarities['Quartile'] = df_viper_with_similarities.apply(get_quartile, axis=1)

# View the 
# Now the DataFrame contains the cosine similarities and quartile information
# Print the final DataFrame
print(df_viper_with_similarities.head())

# Save the final DataFrame with similarities and quartiles to a CSV file
df_viper_with_similarities.to_csv('df_viper_with_quartiles.csv', index=False)


                                    Title  \
0   You’ll Cowards Don’t Even Smoke Crack   
1  You Wanna See Me Dead Cause Of My Hops   
2  Hey, Maybe One Day You’ll See Me Again   
3                         I Sell Dope Boy   
4                            You Gone Die   

                                              Lyrics  \
0  You’ll Cowards Don’t Even Smoke Crack Lyrics[I...   
1  You Wanna See Me Dead Cause Of My Hops Lyrics[...   
2  Hey, Maybe One Day You’ll See Me Again Lyrics[...   
3  I Sell Dope Boy Lyrics[Chorus]\nWhat's up, I d...   
4  You Gone Die Lyrics[Chorus]\nYou gon' die\nYou...   

                                           Embedding        Q1        Q2  \
0  [-0.02860228, 0.030411387, 0.03433824, -0.0099...  0.718014  0.782980   
1  [-0.022501944, 0.031217927, -0.012880615, 0.02...  0.784466  0.805950   
2  [-0.03493272, 0.01452769, -0.004701647, 0.0089...  0.942069  0.696412   
3  [-0.01000656, 0.029114082, -0.004685978, -0.00...  0.746037  0.844312   
4  [-0.05293

## Find Best Match given word input

In [60]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-1.5B-instruct", trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.73it/s]


In [215]:

q1_words = ['aroused', 'astonished', 'excited', 'delighted', 'happy']
q2_words = ['alarmed', 'tense', 'angry', 'afraid', 'annoyed', 'distressed', 'frustrated']
q3_words = ['misreable', 'sad', 'depressed', 'gloomy', 'bored', 'droopy', 'tired']
q4_words = ['pleased', 'glad', 'serene', 'content', 'at ease', 'satisfied', 'relaxed', 'calm']
class_to_words = {
    'Q1': q1_words,
    'Q2': q2_words,
    'Q3': q3_words,
    'Q4': q4_words
}

input_text = 'im feeling happy today'
input_text_embedding = model.encode(input_text)
def compute_cosine_similarities(input_text_embedding, word_class_embedding):
    sim = cosine_similarity([input_text_embedding], [word_class_embedding])[0][0]
    percentage_similarity = sim * 100
    return percentage_similarity

for key in class_to_words:
    string_of_words = ", ".join(class_to_words[key])
    class_to_words[key] = (class_to_words[key] , model.encode(string_of_words))

## get similarity of input text
percentage_sim_dict = {}
for key in class_to_words:
    percentage_sim_dict[key] = compute_cosine_similarities(input_text_embedding, class_to_words[key][1])
print('percentage_sim_dict', percentage_sim_dict)
cosine_values = list(percentage_sim_dict.values())
min_sim = min(cosine_values)
max_sim = max(cosine_values)

# Normalize each cosine similarity to the range [0, 1]
for key in percentage_sim_dict:
    # Perform the min-max normalization
    normalized_similarity = (percentage_sim_dict[key] - min_sim) / (max_sim - min_sim) if max_sim != min_sim else 1.0
    percentage_sim_dict[key] = normalized_similarity

print('original values:', percentage_sim_dict)

percentage_sim_dict {'Q1': np.float32(50.00059), 'Q2': np.float32(36.52183), 'Q3': np.float32(40.318703), 'Q4': np.float32(54.430008)}
original values: {'Q1': np.float32(0.75265956), 'Q2': np.float32(0.0), 'Q3': np.float32(0.21201886), 'Q4': np.float32(1.0)}


In [216]:
sorted_dict = dict(sorted(percentage_sim_dict.items(), key=lambda item: item[1]))
order_of_quartiles = list(sorted_dict.keys())[::-1]
print(order_of_quartiles)

['Q4', 'Q1', 'Q3', 'Q2']


In [217]:
df_viper_with_similarities = pd.read_csv('df_viper_with_quartiles.csv', encoding='utf-8')

def find_best_match(df_viper_with_similarities, percentage_sim_dict, category_weights=None):
    match_list = []  # List to store the matches with their song title and match score
    counter = 0

    for _, row in df_viper_with_similarities.iterrows():
        # Get the song's similarity for each quartile
        song_sim_dict = {
            'Q1': row['Q1'],
            'Q2': row['Q2'],
            'Q3': row['Q3'],
            'Q4': row['Q4']
        }

        match_score = 0
        for key in song_sim_dict:
            # Apply weight to each category similarity comparison
            weight = category_weights.get(key)
            match_score += weight * (percentage_sim_dict[key] - song_sim_dict[key]) ** 2
        # Store the song's match score along with its title
        match_list.append({
            'Song Title': row['Title'],  # Assuming the column name for song title is 'Title'
            'Match Score': match_score
        })
        if counter == 20:
            break
        counter += 1

    # Sort the match list by match score (ascending), best match will be first
    sorted_match_list = sorted(match_list, key=lambda x: x['Match Score'], reverse=False)

    return sorted_match_list

category_weights = {
    order_of_quartiles[0]: 2.5,
    order_of_quartiles[1]: 0,
    order_of_quartiles[2]: 0,
    order_of_quartiles[3]: 0
}

# Call the function to get the sorted matches
sorted_matches = find_best_match(df_viper_with_similarities, percentage_sim_dict, category_weights)
# Print the sorted matches with their song titles and match scores
# print(input_text)
for match in sorted_matches:
    print(f"Song Title: {match['Song Title']}, Match Score: {match['Match Score']}")

Song Title: Hey, Maybe One Day You’ll See Me Again, Match Score: 0.017239078879356384
Song Title: Leanin’ Low, Match Score: 0.10553900897502899
Song Title: Ride Or Die, Match Score: 0.10814587771892548
Song Title: Clip In, Match Score: 0.11444687843322754
Song Title: You Gone Die, Match Score: 0.17485123872756958
Song Title: You Wanna See Me Dead Cause Of My Hops, Match Score: 0.18094801902770996
Song Title: This Is How We Ride, Match Score: 0.2102714627981186
Song Title: You Actin’ Like A Bitch Ass Nigga, Match Score: 0.24874292314052582
Song Title: Can’t Stop Me From Snowbirdin’, Match Score: 0.26634344458580017
Song Title: Feel, Match Score: 0.28880807757377625
Song Title: What I’m Spendin, Match Score: 0.3159124255180359
Song Title: I Gots To Ball, Match Score: 0.34038209915161133
Song Title: I’m A Snake, Match Score: 0.41398897767066956
Song Title: You’ll Cowards Don’t Even Smoke Crack, Match Score: 0.4525252878665924
Song Title: All Dis Money, Match Score: 0.5175617933273315
Song

## Fi

## Get Spotify Song Information

In [56]:
import requests

# Your Spotify API access token (you need to get this via the OAuth flow)
access_token = os.getenv('SPOTIFY_ACCESS_TOKEN')  # Replace with your actual access token

# Song and artist name to search for
track_name = "Shape of You"  # Replace with the name of the song
artist_name = "Ed Sheeran"  # Replace with the artist name

# Prepare the search query (combining track and artist name)
query = f"track:{track_name} artist:{artist_name}"

# API endpoint for searching
url = f"https://api.spotify.com/v1/search?q={query}&type=track&limit=1"  # limit=1 will return the top result

# Set up headers with Authorization (Bearer token)
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Send the GET request to Spotify API
response = requests.get(url, headers=headers)

# Check the response status
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Extract the track ID from the response
    if data['tracks']['items']:
        track_id = data['tracks']['items'][0]['id']
        print(f"Track ID: {track_id}")
    else:
        print("No matching track found.")
else:
    print(f"Error {response.status_code}: {response.text}")


Track ID: 7qiZfU4dY1lWllzX7mPBI3


In [64]:
import requests

# The access token (Bearer token)
access_token = os.getenv('SPOTIFY_ACCESS_TOKEN')

# API endpoint URL
url = f"https://api.spotify.com/v1/audio-features/11dFghVXANMlKmJXsNCbNl"

# Headers to pass the Authorization Bearer token
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Send the GET request to the Spotify API
response = requests.get(url, headers=headers)

# Check the response status code
if response.status_code == 200:
    # If successful, print the response JSON (artist data)
    print(response.json())
else:
    # If the request failed, print the error status and message
    print(f"Error {response.status_code}: {response.text}")


Error 403: {
  "error" : {
    "status" : 403
  }
}


In [ ]:
{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'}, 'href': 'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V', 'id': '6eUKZXaKkcviH0Ku9w2n3V', 'name': 'Ed Sheeran', 'type': 'artist', 'uri': 'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD', 'GW', 'GY', 'HT', 'JM', 'KI', 'LS', 'LR', 'MW', 'MV', 'ML', 'MH', 'FM', 'NA', 'NR', 'NE', 'PW', 'PG', 'WS', 'SM', 'ST', 'SN', 'SC', 'SL', 'SB', 'KN', 'LC', 'VC', 'SR', 'TL', 'TO', 'TT', 'TV', 'VU', 'AZ', 'BN', 'BI', 'KH', 'CM', 'TD', 'KM', 'GQ', 'SZ', 'GA', 'GN', 'KG', 'LA', 'MO', 'MR', 'MN', 'NP', 'RW', 'TG', 'UZ', 'ZW', 'BJ', 'MG', 'MU', 'MZ', 'AO', 'CI', 'DJ', 'ZM', 'CD', 'CG', 'IQ', 'LY', 'TJ', 'VE', 'ET', 'XK'], 'external_urls': {'spotify': 'https://open.spotify.com/album/3T4tUhGYeRNVUGevb0wThu'}, 'href': 'https://api.spotify.com/v1/albums/3T4tUhGYeRNVUGevb0wThu', 'id': '3T4tUhGYeRNVUGevb0wThu', 'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b273ba5db46f4b838ef6027e6f96', 'width': 640, 'height': 640}, {'url': 'https://i.scdn.co/image/ab67616d00001e02ba5db46f4b838ef6027e6f96', 'width': 300, 'height': 300}, {'url': 'https://i.scdn.co/image/ab67616d00004851ba5db46f4b838ef6027e6f96', 'width': 64, 'height': 64}], 'name': '÷ (Deluxe)', 'release_date': '2017-03-03', 'release_date_precision': 'day', 'total_tracks': 16, 'type': 'album', 'uri': 'spotify:album:3T4tUhGYeRNVUGevb0wThu'}, 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'}, 'href': 'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V', 'id': '6eUKZXaKkcviH0Ku9w2n3V', 'name': 'Ed Sheeran', 'type': 'artist', 'uri': 'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD', 'GW', 'GY', 'HT', 'JM', 'KI', 'LS', 'LR', 'MW', 'MV', 'ML', 'MH', 'FM', 'NA', 'NR', 'NE', 'PW', 'PG', 'WS', 'SM', 'ST', 'SN', 'SC', 'SL', 'SB', 'KN', 'LC', 'VC', 'SR', 'TL', 'TO', 'TT', 'TV', 'VU', 'AZ', 'BN', 'BI', 'KH', 'CM', 'TD', 'KM', 'GQ', 'SZ', 'GA', 'GN', 'KG', 'LA', 'MO', 'MR', 'MN', 'NP', 'RW', 'TG', 'UZ', 'ZW', 'BJ', 'MG', 'MU', 'MZ', 'AO', 'CI', 'DJ', 'ZM', 'CD', 'CG', 'IQ', 'LY', 'TJ', 'VE', 'ET', 'XK'], 'disc_number': 1, 'duration_ms': 233712, 'explicit': False, 'external_ids': {'isrc': 'GBAHS1600463'}, 'external_urls': {'spotify': 'https://open.spotify.com/track/7qiZfU4dY1lWllzX7mPBI3'}, 'href': 'https://api.spotify.com/v1/tracks/7qiZfU4dY1lWllzX7mPBI3', 'id': '7qiZfU4dY1lWllzX7mPBI3', 'is_local': False, 'name': 'Shape of You', 'popularity': 86, 'preview_url': None, 'track_number': 4, 'type': 'track', 'uri': 'spotify:track:7qiZfU4dY1lWllzX7mPBI3'}

In [116]:
import requests
input_track_id = None
# Deezer API base URL
DEEZER_API_URL = "https://api.deezer.com/search"

# Function to search for a track by artist and song title
def get_deezer_track_id(artist, track_name):
    params = {
        'q': f'artist:"{artist}" track:"{track_name}"',  # Search by artist and track name
    }

    # Send GET request to Deezer API
    response = requests.get(DEEZER_API_URL, params=params)

    # Check if the response was successful
    if response.status_code == 200:
        data = response.json()
        
        # Check if there are search results
        if data['data']:
            # Extract the first track's ID from the search results
            track = data['data'][0]
            track_id = track['id']
            input_track_id = track['id']
            print('input track id', type(input_track_id))
            print(f"Track ID for '{track_name}' by {artist}: {track_id}")
            return track_id
        else:
            print(f"No results found for '{track_name}' by {artist}.")
    else:
        print(f"Error: {response.status_code}")

# Example usage
input_track_id = get_deezer_track_id("Viper", "I'm a snake")
print('input track id', input_track_id)


input track id <class 'int'>
Track ID for 'I'm a snake' by Viper: 1784783527
input track id 1784783527


In [117]:
import requests

# Deezer API base URL for track details
DEEZER_TRACK_URL = "https://api.deezer.com/track/"

def get_track_details(track_id):
    # Send GET request to Deezer API to fetch track details
    response = requests.get(f"{DEEZER_TRACK_URL}{track_id}")

    # Check if the response was successful
    if response.status_code == 200:
        track_info = response.json()

        # Extract information from the response
        track_name = track_info.get('title', 'N/A')
        artist_name = track_info.get('artist', {}).get('name', 'N/A')
        album_name = track_info.get('album', {}).get('title', 'N/A')
        bpm = track_info.get('bpm', 'BPM not available')  # BPM of the track
        genre = track_info.get('genres', {}).get('data', [{}])[0].get('name', 'N/A')  # Genre
        duration = track_info.get('duration', 'N/A')  # Duration in seconds
        release_date = track_info.get('release_date', 'N/A')  # Release date

        # Print the track details
        print(f"Track: {track_name}")
        print(f"Artist: {artist_name}")
        print(f"Album: {album_name}")
        print(f"BPM: {bpm}")
        print(f"Genre: {genre}")
        print(f"Duration (seconds): {duration}")
        print(f"Release Date: {release_date}")
    else:
        print(f"Error: {response.status_code}")

# Example usage with the track ID obtained earlier
print(input_track_id)
get_track_details(str(int(input_track_id)))  # Replace with the actual track ID (e.g., for "Blinding Lights")


1784783527
Track: I'm A Snake In The Grass (Money Rolls Remix)
Artist: Viper
Album: The Hiram Clarke Hustler
BPM: 0
Genre: N/A
Duration (seconds): 204
Release Date: 2008-01-01


In [125]:
import requests

# Your API key from AudioDB
APIKEY = 'EADFE'

# Base URL for the AudioDB API
BASE_URL = f'www.theaudiodb.com/api/v1/json/{APIKEY}/track.php?m=2115888'

# Function to search for a track
def get_track_details(track_name, track_title):
    # Search URL for track
    # search_url = f'{BASE_URL}searchtrack.php?s={track_name}&t={track_title}'
    search_url = BASE_URL
    # Send GET request to the AudioDB API
    response = requests.get(search_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Check if any tracks are found
        if data.get('track', []):
            track = data['track'][0]
            track_bpm = track.get('intbpm', 'BPM not available')
            genre = track.get('strGenre', 'Genre not available')
            album = track.get('strAlbum', 'Album not available')
            artist = track.get('strArtist', 'Artist not available')
            
            print(f"Track: {track_title}")
            print(f"Artist: {artist}")
            print(f"Album: {album}")
            print(f"BPM: {track_bpm}")
            print(f"Genre: {genre}")
        else:
            print(f"No results found for '{track_title}' by '{track_name}'.")
    else:
        print(f"Error: {response.status_code}")

# Example usage
get_track_details('Ed Sheeran', 'Shape of You')


MissingSchema: Invalid URL 'www.theaudiodb.com/api/v1/json/EADFE/track.php?m=2115888': No scheme supplied. Perhaps you meant https://www.theaudiodb.com/api/v1/json/EADFE/track.php?m=2115888?